# Работа с ORM в SQL Alchemy 

<a target="_blank" href="https://colab.research.google.com/github/sozykin/middle_python/blob/main/06/06_orm.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy import Column
from sqlalchemy import String, Integer

## Создание класса с отображением в таблицу базы данных

In [ ]:
# Базовый класс для создание классов, которые будут сохраняться в базе данных
Base = declarative_base()

In [ ]:
# Создаем класс и описываем отображение атрибутов в столбцы таблицы базы данных
class Name(Base):
    # Название таблицы в базе данных
    __tablename__ = 'names'

    # Атрибуты  
    id = Column('id', Integer)
    name = Column('name', String)
    number_of_persons = Column('number_of_persons', Integer)
    global_id = Column('global_id', Integer, primary_key=True)
    year = Column('year', Integer)
    month = Column('month', String)

    # Функция инициализации
    def __init__(self, id, name, number_of_persons, global_id, year, month):
        self.id = id
        self.name = name
        self.number_of_persons = number_of_persons
        self.global_id = global_id
        self.year = year
        self.month = month

    # Текстовое представление объекта
    def __repr__(self):
        return(f"{self.id}, {self.name}, {self.number_of_persons}, {self.global_id}, {self.year}, {self.month}")

In [ ]:
# Создаем соединение с базой данных SQLite
engine = create_engine("sqlite:///test.db", echo=True)

In [ ]:
# Создаем таблицы в базе данных
Base.metadata.create_all(bind=engine)

## Создаем объекты класса и сохраняем их в базе данных

In [ ]:
# Создаем сессию
Sesion = sessionmaker(bind=engine)
session = Sesion()

In [ ]:
# Создаем новый объект класса Name
name1 = Name(3, 'Анна', 190, 37750256, 2015, 'январь')

In [ ]:
# Сохраняем объект в базе данных
session.add(name1)
session.commit()

In [ ]:
# Создаем еще один объект
name2 = Name(4, 'Варвара', 190, 37750257, 2015, 'январь')

In [ ]:
# Сохраняем объект в базе данных
session.add(name2)
session.commit()

## Поиск объектов в базе

In [ ]:
# Извлекаем из базы нужные нам объекты
results = session.query(Name).filter(Name.name == 'Варвара')

In [ ]:
for row in results:
    print(row)

In [ ]:
session.close()

## Современный декларативный подход к описанию отображение атрибутов объекта в поля таблицы

In [ ]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import Session

In [ ]:
# Базовый класс для создание классов, которые будут сохраняться в базе данных
class Base(DeclarativeBase):
    pass

In [ ]:
# Создаем класс и описываем отображение атрибутов в столбцы таблицы базы данных
class Name2(Base):
    # Название таблицы в базе данных
    __tablename__ = 'names2'

    # Атрибуты  
    id: Mapped[int] 
    name: Mapped[str]
    number_of_persons: Mapped[str] 
    global_id: Mapped[int] = mapped_column(primary_key=True)
    year: Mapped[int]
    month: Mapped[str]

    # Текстовое представление объекта
    def __repr__(self):
        return(f"{self.id}, {self.name}, {self.number_of_persons}, {self.global_id}, {self.year}, {self.month}")

In [ ]:
# Создаем таблицу
Base.metadata.create_all(engine)

## Создаем объекты

In [ ]:
# Метод __init__ с именованными аргументами был сгенерирован автоматически
new_name = Name2(id=3,
                 name='Анна', 
                 number_of_persons=190, 
                 global_id=37750256,
                 year=2015, 
                 month='январь')

In [ ]:
new_name

In [ ]:
new_name2 = Name2(id=4,
                 name='Варвара', 
                 number_of_persons=190, 
                 global_id=37750257,
                 year=2015, 
                 month='январь')

## Сохраняем данные в базе

In [ ]:
with Session(engine) as session:
    session.add(new_name)
    session.add(new_name2)
    session.commit()

## Извлекаем данные из базы

In [ ]:
from sqlalchemy import select

In [ ]:
with Session(engine) as session:
    results = session.execute(select(Name2))
    for row in results:
        print(row)

In [ ]:
with Session(engine) as session:
    results = session.execute(select(Name2).where(Name2.name == 'Анна'))
    for row in results:
        print(row)

## Изменяем данные в базе

In [ ]:
session = Session(engine)

In [ ]:
name = session.execute(select(Name2).where(Name2.name == 'Анна')).scalar_one()

In [ ]:
name

In [ ]:
name.name = 'Мария'

In [ ]:
session.dirty

In [ ]:
session.commit()

In [ ]:
session.dirty

In [ ]:
session.close()

## Удаляем данные из базы

In [ ]:
with Session(engine) as session:
    session.delete(new_name)
    session.delete(new_name2)
    session.commit()

## Загрузка данных из JSON а базу с помощью SQL Alchemy

In [ ]:
import json

In [ ]:
with open("names_f.json", "r") as read_file:
    data = json.load(read_file)

In [ ]:
data

## Записываем данные в базу

In [ ]:
with Session(engine) as session:
    for record in data:
        new_name = Name2(id=record['ID'],
                         name=record['Name'], 
                         number_of_persons=record['NumberOfPersons'], 
                         global_id=record['global_id'],
                         year=record['Year'], 
                         month=record['Month'])
        session.add(new_name)
    session.commit()